In [ ]:
import torch
print("GPU Available:", torch.cuda.is_available())


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install mtcnn
!pip install dlib
!pip install opencv-python
!pip install opencv-contrib-python

!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.4 MB/s eta 0:00:00
--2024-10-08 12:37:22--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  18.7MB/s    in 3.3s    

2024-10-08 12:37:26 (18.7 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
from pathlib import Path
video_path = Path('/content/drive/MyDrive/Personal/BUDKA/Copy of 07.mp4')
video_path = Path('/content/drive/MyDrive/Personal/BUDKA/cropped_short.mp4')
video_path.exists()

True

In [ ]:
import cv2
import dlib
import numpy as np
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt


In [ ]:
# Initialize MTCNN for face detection and Dlib for facial landmark detection
detector = MTCNN(device='GPU:0')  # Face detector
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')  # Download the file if needed

# Open video
cap = cv2.VideoCapture(video_path)


In [ ]:
def extract_landmarks(frame, faces):
    # Convert to grayscale for dlib
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # List to store landmark positions for each face
    landmarks_list = []

    for face in faces:
        x, y, w, h = face['box']
        rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))

        # Get the landmarks for the face
        landmarks = landmark_predictor(gray, rect)

        # Extract the (x, y) coordinates of the eyes, brows, and nose
        coords = np.zeros((68, 2), dtype="int")
        for i in range(0, 68):
            coords[i] = (landmarks.part(i).x, landmarks.part(i).y)

        landmarks_list.append(coords)

    return landmarks_list

In [ ]:
# def crop_eye_nose_brow(frame, landmarks):
def crop_eye_brow(frame, landmarks):
    # Points for the left eye, right eye, brows, and nose bridge
    eye_brow_points = landmarks[17:27]  # Brows
    eye_points = landmarks[36:48]       # Eyes
    # nose_bridge = landmarks[27:31]      # Nose bridge

    # Combine the points and create a bounding box
    # all_points = np.concatenate((eye_brow_points, eye_points, nose_bridge))
    all_points = np.concatenate((eye_brow_points, eye_points))

    x, y, w, h = cv2.boundingRect(all_points)

    # Add some padding around the region
    padding = 20
    x = max(0, x - padding)
    y = max(0, y - padding)
    w = w + 2 * padding
    h = h + 2 * padding

    # Crop the region of interest (ROI)
    cropped = frame[y:y + h, x:x + w]

    return cropped


In [ ]:
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and output video format
out = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Detect faces in the frame
    faces = detector.detect_faces(frame,
                                  fit_to_image=False,
                                  limit_boundaries_landmarks=True,
                                  min_face_size=400,  # default 20
                                  min_size=20,  # default min size of smallest scale
                                  scale_factor=0.709,  # default. Increase
                                  threshold_pnet=0.6,  # default Up

                                  threshold_rnet=0.85,  # default
                                  threshold_onet=0.6,  # default

                                  nms_pnet=0.7,
                                  nps_pnet2=0.8,

                                  steps_factor=0.5,  # default
                                  # output_type='numpy'
                                  )

    if len(faces) > 0:
        # Extract landmarks for the detected faces
        landmarks_list = extract_landmarks(frame, faces)

        for landmarks in landmarks_list:
            # Crop the eye, nose, and brow area
            cropped = crop_eye_brow(frame, landmarks)

            # Optionally, display the cropped region
            # cv2.imshow('Cropped Region', cropped)
            # cv2.waitKey(1)  # Display for 1 ms

    # Write frame to the output video
    out.write(frame)

# Release video objects
cap.release()
out.release()
cv2.destroyAllWindows()


In [ ]:
output_video_path = '/content/output_video.mp4'  # Local path in Colab
drive_output_path = '/content/drive/MyDrive/Personal/BUDKA/output_video_short.mp4'  # Path in Google Drive

# Move the video to Google Drive
!cp {output_video_path} {drive_output_path}
